In [1]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import gc
import time
import subprocess
from concurrent.futures import ProcessPoolExecutor, as_completed

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, Draw
from rdkit import RDConfig
from rdkit.Chem import Descriptors, rdMolDescriptors, Lipinski, rdDistGeom, rdPartialCharges
from rdkit.Chem.AllChem import GetMorganGenerator
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.models import model_from_json

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error,mean_squared_error

In [ ]:
tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [6]:
target_path = "result/2_solubility_fingerprint_compare"
os.makedirs(target_path, exist_ok=True)

In [7]:
data_ws = pd.read_csv('./data/ws496_logS.csv', dtype={'SMILES': 'string'})
smiles_ws = data_ws['SMILES']
y_ws = data_ws.iloc[:, 2]

data_delaney = pd.read_csv('./data/delaney-processed.csv', dtype={'smiles': 'string'})
smiles_de = data_delaney['smiles']
y_de = data_delaney.iloc[:, 1]

data_lovric2020 = pd.read_csv('./data/Lovric2020_logS0.csv', dtype={'isomeric_smiles': 'string'})
smiles_lo = data_lovric2020['isomeric_smiles']
y_lo = data_lovric2020.iloc[:, 1]

data_huuskonen = pd.read_csv('./data/huusk.csv', dtype={'SMILES': 'string'})
smiles_hu = data_huuskonen['SMILES']
y_hu = data_huuskonen.iloc[:, -1].astype('float')

In [8]:
def mol3d(mol):
    mol = Chem.AddHs(mol)
    optimization_methods = [
        (AllChem.EmbedMolecule, (mol, AllChem.ETKDGv3()), {}),
        (AllChem.UFFOptimizeMolecule, (mol,), {'maxIters': 200}),
        (AllChem.MMFFOptimizeMolecule, (mol,), {'maxIters': 200})
    ]

    for method, args, kwargs in optimization_methods:
        try:
            method(*args, **kwargs)
            if mol.GetNumConformers() > 0:
                return mol
        except ValueError as e:
            print(f"Error: {e} - Trying next optimization method [{method}]")

    print(f"Invalid mol for 3d {'\033[94m'}{Chem.MolToSmiles(mol)}{'\033[0m'} - No conformer generated")
    return None

In [9]:
def convert_smiles_to_mol(smiles, fail_folder=None, index=None, yvalue=None):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"[convert_smiles_to_mol] Cannot convert {smiles} to Mols")
        return None, {"smiles": smiles, "y_value": yvalue, "error": "Invalid SMILES"}

    try:
        Chem.Kekulize(mol, clearAromaticFlags=True)
        isomeric_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
        mol = Chem.MolFromSmiles(isomeric_smiles)
    except Exception as e:
        print(f"[convert_smiles_to_mol] failed {smiles} isomeric_smiles by {e}")
        if fail_folder and index is not None:
            img_path = os.path.join(fail_folder, f"mol_{index}.png")
            img = Draw.MolToImage(mol)
            img.save(img_path)
        return None, {"smiles": smiles, "y_value": yvalue, "error": f"Isomeric SMILES error: {e}"}

    try:
        Chem.SanitizeMol(mol)
    except Exception as e:
        print(f"[convert_smiles_to_mol] failed {smiles} SanitizeMol by {e}")
        if fail_folder and index is not None:
            img_path = os.path.join(fail_folder, f"mol_{index}.png")
            img = Draw.MolToImage(mol)
            img.save(img_path)
        return None, {"smiles": smiles, "y_value": yvalue, "error": f"SanitizeMol error: {e}"}

    return mol, None

In [10]:
def process_smiles(smiles, yvalue, fail_folder, index):
    mol, error = convert_smiles_to_mol(smiles, fail_folder, index, yvalue)
    if error:
        return None, None, error

    mol_3d = mol3d(mol)
    if mol_3d:
        return smiles, yvalue, None
    else:
        img_path = os.path.join(fail_folder, f"mol_{index}.png")
        img = Draw.MolToImage(mol)
        img.save(img_path)
        return None, None, {"smiles": smiles, "y_value": yvalue}

def process_dataset(smiles_list, y_values, dataset_name, target_path="result", max_workers=None):
    start = time.time()
    valid_smiles, valid_y = [], []
    error_smiles_list = []
    fail_folder = f"{target_path}/failed/{dataset_name}"
    os.makedirs(fail_folder, exist_ok=True)

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_smiles, smiles, yvalue, fail_folder, i)
            for i, (smiles, yvalue) in enumerate(zip(smiles_list, y_values))
        ]
        for future in as_completed(futures):
            smiles, yvalue, error = future.result()
            if error:
                error_smiles_list.append(error)
            elif smiles is not None and yvalue is not None:
                valid_smiles.append(smiles)
                valid_y.append(yvalue)

    if error_smiles_list:
        error_df = pd.DataFrame(error_smiles_list)
        error_df.to_csv(os.path.join(fail_folder, "failed_smiles.csv"), index=False)
    print(f" [{dataset_name:<10}] : {time.time()-start:.4f} sec")
    return valid_smiles, valid_y

In [ ]:
smiles_ws, y_ws = process_dataset(smiles_ws, y_ws, "ws496", target_path)
smiles_de, y_de = process_dataset(smiles_de, y_de, "delaney", target_path)
smiles_lo, y_lo = process_dataset(smiles_lo, y_lo, "Lovric2020_logS0", target_path)
smiles_hu, y_hu = process_dataset(smiles_hu, y_hu, "huusk", target_path)

In [12]:
LEN_OF_FF = 2048
LEN_OF_MA = 167
LEN_OF_AV = 512

In [13]:
def get_fingerprints(mol):
    if mol is None:
        return None, None, None
    
    morgan_generator = GetMorganGenerator(radius=2, fpSize=LEN_OF_FF)
    ecfp = morgan_generator.GetFingerprint(mol)
    ecfp_array = np.zeros((LEN_OF_FF,),dtype=int)
    DataStructs.ConvertToNumpyArray(ecfp, ecfp_array)
    
    maccs = Chem.rdMolDescriptors.GetMACCSKeysFingerprint(mol)

    avalon_fp = GetAvalonFP(mol)
    avalon_array = np.zeros((LEN_OF_AV,),dtype=int)
    DataStructs.ConvertToNumpyArray(avalon_fp, avalon_array)
    
    return ecfp_array, maccs, avalon_array

def fp_converter(data, use_parallel=True):
    mols = [Chem.MolFromSmiles(smi) for smi in data]
    
    if use_parallel:
        try:            
            with ProcessPoolExecutor() as executor:
                results = list(executor.map(get_fingerprints, mols))
        except Exception as e:
            print(f"Parallel processing failed due to: {e}. Falling back to sequential processing.")
            use_parallel = False
    
    if not use_parallel:
        results = [get_fingerprints(mol) for mol in mols]
    
    ECFP, MACCS, AvalonFP = zip(*results)
    
    ECFP_container = np.vstack([arr for arr in ECFP if arr is not None])
    MACCS_container = np.zeros((len(MACCS), LEN_OF_MA), dtype=int)
    AvalonFP_container = np.vstack([arr for arr in AvalonFP if arr is not None])

    for i, fp in enumerate(MACCS):
        if fp is not None:
            DataStructs.ConvertToNumpyArray(fp, MACCS_container[i])
    
    return mols, ECFP_container, MACCS_container, AvalonFP_container

In [14]:
mol_ws, x_ws, MACCS_ws, AvalonFP_ws = fp_converter(smiles_ws)
mol_de, x_de, MACCS_de, AvalonFP_de = fp_converter(smiles_de)
mol_lo, x_lo, MACCS_lo, AvalonFP_lo = fp_converter(smiles_lo)
mol_hu, x_hu, MACCS_hu, AvalonFP_hu = fp_converter(smiles_hu)

In [15]:
#Morgan Fingerprint (ECFP)
xtr_ws1, xte_ws1, ytr_ws1, yte_ws1 = train_test_split(x_ws, y_ws, test_size=0.2,random_state=42)
xtr_de1, xte_de1, ytr_de1, yte_de1 = train_test_split(x_de, y_de, test_size=0.2,random_state=42)
xtr_lo1, xte_lo1, ytr_lo1, yte_lo1 = train_test_split(x_lo, y_lo, test_size=0.2,random_state=42)
xtr_hu1, xte_hu1, ytr_hu1, yte_hu1 = train_test_split(x_hu, y_hu, test_size=0.2,random_state=42)

In [16]:
#MACCS Fingerprint
xtr_ws2, xte_ws2, ytr_ws2, yte_ws2 = train_test_split(MACCS_ws, y_ws, test_size=0.2,random_state=42)
xtr_de2, xte_de2, ytr_de2, yte_de2 = train_test_split(MACCS_de, y_de, test_size=0.2,random_state=42)
xtr_lo2, xte_lo2, ytr_lo2, yte_lo2 = train_test_split(MACCS_lo, y_lo, test_size=0.2,random_state=42)
xtr_hu2, xte_hu2, ytr_hu2, yte_hu2 = train_test_split(MACCS_hu, y_hu, test_size=0.2,random_state=42)

In [17]:
#Avalon Fingerprint
xtr_ws3, xte_ws3, ytr_ws3, yte_ws3 = train_test_split(AvalonFP_ws, y_ws, test_size=0.2,random_state=42)
xtr_de3, xte_de3, ytr_de3, yte_de3 = train_test_split(AvalonFP_de, y_de, test_size=0.2,random_state=42)
xtr_lo3, xte_lo3, ytr_lo3, yte_lo3 = train_test_split(AvalonFP_lo, y_lo, test_size=0.2,random_state=42)
xtr_hu3, xte_hu3, ytr_hu3, yte_hu3 = train_test_split(AvalonFP_hu, y_hu, test_size=0.2,random_state=42)

In [18]:
def concatenate_to_numpy(*dataframes):
    numpy_arrays = [df.to_numpy() if isinstance(df, pd.DataFrame) else df for df in dataframes]
    if not all(isinstance(arr, np.ndarray) for arr in numpy_arrays):
        raise ValueError("All inputs must be either pandas DataFrame or numpy array")
    return np.concatenate(numpy_arrays, axis=1)

In [19]:
#ECFP + MACCS
group_nws1 = concatenate_to_numpy(x_ws, MACCS_ws)
group_nde1 = concatenate_to_numpy(x_de, MACCS_de)
group_nlo1 = concatenate_to_numpy(x_lo, MACCS_lo)
group_nhu1 = concatenate_to_numpy(x_hu, MACCS_hu)

xtr_ws12, xte_ws12, ytr_ws12, yte_ws12 = train_test_split(group_nws1, y_ws, test_size=0.2,random_state=42)
xtr_de12, xte_de12, ytr_de12, yte_de12 = train_test_split(group_nde1, y_de, test_size=0.2,random_state=42)
xtr_lo12, xte_lo12, ytr_lo12, yte_lo12 = train_test_split(group_nlo1, y_lo, test_size=0.2,random_state=42)
xtr_hu12, xte_hu12, ytr_hu12, yte_hu12 = train_test_split(group_nhu1, y_hu, test_size=0.2,random_state=42)

In [20]:
#ECFP + Avalon
group_nws2 = concatenate_to_numpy(x_ws, AvalonFP_ws)
group_nde2 = concatenate_to_numpy(x_de, AvalonFP_de)
group_nlo2 = concatenate_to_numpy(x_lo, AvalonFP_lo)
group_nhu2 = concatenate_to_numpy(x_hu, AvalonFP_hu)

xtr_ws13, xte_ws13, ytr_ws13, yte_ws13 = train_test_split(group_nws2, y_ws, test_size=0.2,random_state=42)
xtr_de13, xte_de13, ytr_de13, yte_de13 = train_test_split(group_nde2, y_de, test_size=0.2,random_state=42)
xtr_lo13, xte_lo13, ytr_lo13, yte_lo13 = train_test_split(group_nlo2, y_lo, test_size=0.2,random_state=42)
xtr_hu13, xte_hu13, ytr_hu13, yte_hu13 = train_test_split(group_nhu2, y_hu, test_size=0.2,random_state=42)

In [21]:
#MACCS + Avalon
group_nws3 = concatenate_to_numpy(MACCS_ws, AvalonFP_ws)
group_nde3 = concatenate_to_numpy(MACCS_de, AvalonFP_de)
group_nlo3 = concatenate_to_numpy(MACCS_lo, AvalonFP_lo)
group_nhu3 = concatenate_to_numpy(MACCS_hu, AvalonFP_hu)

xtr_ws23, xte_ws23, ytr_ws23, yte_ws23 = train_test_split(group_nws3, y_ws, test_size=0.2,random_state=42)
xtr_de23, xte_de23, ytr_de23, yte_de23 = train_test_split(group_nde3, y_de, test_size=0.2,random_state=42)
xtr_lo23, xte_lo23, ytr_lo23, yte_lo23 = train_test_split(group_nlo3, y_lo, test_size=0.2,random_state=42)
xtr_hu23, xte_hu23, ytr_hu23, yte_hu23 = train_test_split(group_nhu3, y_hu, test_size=0.2,random_state=42)

In [22]:
#ECFP + MACCS + Avalon
fgroup_nws = concatenate_to_numpy(x_ws, MACCS_ws, AvalonFP_ws)
fgroup_nde = concatenate_to_numpy(x_de, MACCS_de, AvalonFP_de)
fgroup_nlo = concatenate_to_numpy(x_lo, MACCS_lo, AvalonFP_lo)
fgroup_nhu = concatenate_to_numpy(x_hu, MACCS_hu, AvalonFP_hu)

xtr_wsf, xte_wsf, ytr_wsf, yte_wsf = train_test_split(fgroup_nws, y_ws, test_size=0.2,random_state=42)
xtr_def, xte_def, ytr_def, yte_def = train_test_split(fgroup_nde, y_de, test_size=0.2,random_state=42)
xtr_lof, xte_lof, ytr_lof, yte_lof = train_test_split(fgroup_nlo, y_lo, test_size=0.2,random_state=42)
xtr_huf, xte_huf, ytr_huf, yte_huf = train_test_split(fgroup_nhu, y_hu, test_size=0.2,random_state=42)

In [23]:
BATCHSIZE = 32
EPOCHS = 100
lr = 0.0001
decay = 1e-4

In [24]:
def new_inference_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(
            units=1024,
            activation='relu',
            kernel_initializer='glorot_uniform',
            kernel_regularizer=tf.keras.regularizers.l2(decay)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(
            units=469,
            activation='relu',
            kernel_initializer='glorot_uniform',
            kernel_regularizer=tf.keras.regularizers.l2(decay)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=1)
    ])
    
    return model

def save_model():
    model = new_inference_model()
    model_json = model.to_json()
    os.makedirs("save_model", exist_ok=True)  # Ensure the directory exists
    with open("save_model/model_config.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("save_model/model_weights.weights.h5")

In [25]:
# Environment settings for optimal performance
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow INFO and WARNING messages
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 --tf_xla_enable_xla_devices'
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda --xla_gpu_force_compilation_parallelism=1'

import logging
class FilterNUMA(logging.Filter):
    def filter(self, record):
        return "NUMA" not in record.getMessage() and "XLA service" not in record.getMessage()

logger = logging.getLogger()
logger.setLevel(logging.ERROR)
for handler in logger.handlers:
    handler.addFilter(FilterNUMA())

In [26]:
import re
import subprocess
from tensorflow.keras.models import load_model

In [27]:
def model_study_datasets(xtr_list, ytr_list):
    models = []
    
    fps_file = 'new_fps.npy'
    y_true_file = 'y_true.npy'
    
    for i in range(len(xtr_list)):
        np.save(fps_file, xtr_list[i])
        np.save(y_true_file, ytr_list[i])
        
        save_model()
        
        result = subprocess.run(['python3', './extra_code/basic_model.py', 
                                 str(BATCHSIZE), str(EPOCHS),
                                 fps_file, y_true_file],
                                stdout=subprocess.PIPE, 
                                stderr=subprocess.PIPE, 
                                text=True)
    
        if result.stderr:
            sys.stderr.write(result.stderr)
        
        warning_patterns = [
            r'WARNING: All log messages before absl::InitializeLog\(\) is called are written to STDERR',
            r'could not open file to read NUMA node',
            r'Your kernel may have been built without NUMA support',
            r'XLA service .* initialized for platform (CUDA|Host)',
            r'StreamExecutor device \(0\):',
            r'Created device /job:localhost/replica:0/task:0/device:GPU:0',
            r'Using CUDA malloc Async allocator for GPU',
            r'Could not identify NUMA node of platform GPU',
        ]

        filtered_stderr = "\n".join([
            line for line in result.stderr.splitlines()
            if not any(re.search(pattern, line) for pattern in warning_patterns)
        ])
        
        if filtered_stderr:
            print(f"[{i}] Filtered stderr:")
            print(filtered_stderr)
        print(f"Return code: {result.returncode}")

        if result.returncode != 0 and filtered_stderr and not all('Your kernel may have been built without NUMA support' in line for line in filtered_stderr.split('\n')):
            raise ValueError(f"[{i}] Error during learning result process: {filtered_stderr}")
        
        try:
            trained_model = tf.keras.models.load_model('save_model/trained_model.keras')
            models.append(trained_model)
            print(f"Model {i+1} loaded successfully")
        except Exception as e:
            print(f"[{i}] Error loading model: {str(e)}")
            continue

    os.remove(fps_file)
    os.remove(y_true_file)
    
    return models

In [ ]:
xtr_ws=[xtr_ws1, xtr_ws2, xtr_ws3, xtr_ws12, xtr_ws13, xtr_ws23, xtr_wsf]
ytr_ws=[ytr_ws1, ytr_ws2, ytr_ws3, ytr_ws12, ytr_ws13, ytr_ws23, ytr_wsf]
xte_ws=[xte_ws1, xte_ws2, xte_ws3, xte_ws12, xte_ws13, xte_ws23, xte_wsf]
yte_ws=[yte_ws1, yte_ws2, yte_ws3, yte_ws12, yte_ws13, yte_ws23, yte_wsf]
res_ws = model_study_datasets(xtr_ws,ytr_ws)
# 7m 30s

In [ ]:
xtr_de=[xtr_de1, xtr_de2, xtr_de3, xtr_de12, xtr_de13, xtr_de23, xtr_def]
ytr_de=[ytr_de1, ytr_de2, ytr_de3, ytr_de12, ytr_de13, ytr_de23, ytr_def]
xte_de=[xte_de1, xte_de2, xte_de3, xte_de12, xte_de13, xte_de23, xte_def]
yte_de=[yte_de1, yte_de2, yte_de3, yte_de12, yte_de13, yte_de23, yte_def]
res_de = model_study_datasets(xtr_de,ytr_de)
#12m 43s

In [ ]:
xtr_lo=[xtr_lo1, xtr_lo2, xtr_lo3, xtr_lo12, xtr_lo13, xtr_lo23, xtr_lof]
ytr_lo=[ytr_lo1, ytr_lo2, ytr_lo3, ytr_lo12, ytr_lo13, ytr_lo23, ytr_lof]
xte_lo=[xte_lo1, xte_lo2, xte_lo3, xte_lo12, xte_lo13, xte_lo23, xte_lof]
yte_lo=[yte_lo1, yte_lo2, yte_lo3, yte_lo12, yte_lo13, yte_lo23, yte_lof]
res_lo = model_study_datasets(xtr_lo,ytr_lo)
#10m 9.2s

In [ ]:
xtr_hu=[xtr_hu1, xtr_hu2, xtr_hu3, xtr_hu12, xtr_hu13, xtr_hu23, xtr_huf]
ytr_hu=[ytr_hu1, ytr_hu2, ytr_hu3, ytr_hu12, ytr_hu13, ytr_hu23, ytr_huf]
xte_hu=[xte_hu1, xte_hu2, xte_hu3, xte_hu12, xte_hu13, xte_hu23, xte_huf]
yte_hu=[yte_hu1, yte_hu2, yte_hu3, yte_hu12, yte_hu13, yte_hu23, yte_huf]
res_hu = model_study_datasets(xtr_hu,ytr_hu)
#13m 24s

In [32]:
def print_metrics(dataset, model_name, ypre,yte):
    r2 = r2_score(yte,ypre)
    mae  = mean_absolute_error(yte,ypre)
    mse  = mean_squared_error(yte,ypre)
    rmse = root_mean_squared_error(yte,ypre)
    print(f"[data : {dataset} ][model : {model_name} ] = r2 : {r2:.5f}, mae : {mae:.5f}, mse : {mse:.5f}, rmse : {rmse:.5f}")

In [ ]:
print_metrics('ws','ECEP      ', res_ws[0].predict(xte_ws[0], verbose=0), yte_ws[0])
print_metrics('ws','MACCS     ', res_ws[1].predict(xte_ws[1], verbose=0), yte_ws[1])
print_metrics('ws','Avalon    ', res_ws[2].predict(xte_ws[2], verbose=0), yte_ws[2])
print_metrics('ws','ECEP_MACCS', res_ws[3].predict(xte_ws[3], verbose=0), yte_ws[3])
print_metrics('ws','ECEP_Ava. ', res_ws[4].predict(xte_ws[4], verbose=0), yte_ws[4])
print_metrics('ws','MACCS_Ava.', res_ws[5].predict(xte_ws[5], verbose=0), yte_ws[5])
print_metrics('ws','ECFP+MACCS+Ava.', res_ws[6].predict(xte_ws[6], verbose=0), yte_ws[6])

# [data : ws ][model : ECEP       ] = r2 : 0.66232, mae : 0.83593, mse : 1.10171, rmse : 1.04962
# [data : ws ][model : MACCS      ] = r2 : 0.74583, mae : 0.71737, mse : 0.82926, rmse : 0.91064
# [data : ws ][model : Avalon     ] = r2 : 0.71877, mae : 0.75613, mse : 0.91754, rmse : 0.95788
# [data : ws ][model : ECEP_MACCS ] = r2 : 0.77450, mae : 0.63215, mse : 0.73572, rmse : 0.85774
# [data : ws ][model : ECEP_Ava.  ] = r2 : 0.74313, mae : 0.67417, mse : 0.83806, rmse : 0.91546
# [data : ws ][model : MACCS_Ava. ] = r2 : 0.77575, mae : 0.65553, mse : 0.73164, rmse : 0.85536
# [data : ws ][model : ECFP+MACCS+Ava. ] = r2 : 0.78273, mae : 0.62530, mse : 0.70886, rmse : 0.84194

In [ ]:
print_metrics('de','ECEP      ', res_de[0].predict(xte_de[0], verbose=0), yte_de[0])
print_metrics('de','MACCS     ', res_de[1].predict(xte_de[1], verbose=0), yte_de[1])
print_metrics('de','Avalon    ', res_de[2].predict(xte_de[2], verbose=0), yte_de[2])
print_metrics('de','ECEP_MACCS', res_de[3].predict(xte_de[3], verbose=0), yte_de[3])
print_metrics('de','ECEP_Ava. ', res_de[4].predict(xte_de[4], verbose=0), yte_de[4])
print_metrics('de','MACCS_Ava.', res_de[5].predict(xte_de[5], verbose=0), yte_de[5])
print_metrics('de','ECFP+MACCS+Ava.', res_de[6].predict(xte_de[6], verbose=0), yte_de[6])

# [data : de ][model : ECEP       ] = r2 : 0.83667, mae : 0.46861, mse : 0.48737, rmse : 0.69812
# [data : de ][model : MACCS      ] = r2 : 0.79751, mae : 0.49282, mse : 0.60425, rmse : 0.77734
# [data : de ][model : Avalon     ] = r2 : 0.85147, mae : 0.41674, mse : 0.44323, rmse : 0.66576
# [data : de ][model : ECEP_MACCS ] = r2 : 0.85653, mae : 0.38113, mse : 0.42812, rmse : 0.65431
# [data : de ][model : ECEP_Ava.  ] = r2 : 0.88713, mae : 0.35384, mse : 0.33680, rmse : 0.58035
# [data : de ][model : MACCS_Ava. ] = r2 : 0.88132, mae : 0.34731, mse : 0.35414, rmse : 0.59509
# [data : de ][model : ECFP+MACCS+Ava. ] = r2 : 0.88028, mae : 0.34027, mse : 0.35725, rmse : 0.59770

# [data : de ][model : ECEP       ] = r2 : 0.83681, mae : 0.48289, mse : 0.48696, rmse : 0.69783
# [data : de ][model : MACCS      ] = r2 : 0.81288, mae : 0.46656, mse : 0.55837, rmse : 0.74724
# [data : de ][model : Avalon     ] = r2 : 0.86393, mae : 0.38842, mse : 0.40602, rmse : 0.63720
# [data : de ][model : ECEP_MACCS ] = r2 : 0.86069, mae : 0.38990, mse : 0.41570, rmse : 0.64475
# [data : de ][model : ECEP_Ava.  ] = r2 : 0.86526, mae : 0.39938, mse : 0.40208, rmse : 0.63410
# [data : de ][model : MACCS_Ava. ] = r2 : 0.87254, mae : 0.36285, mse : 0.38035, rmse : 0.61673
# [data : de ][model : ECFP+MACCS+Ava. ] = r2 : 0.89274, mae : 0.32713, mse : 0.32007, rmse : 0.56575

In [ ]:
print_metrics('lo','ECEP      ', res_lo[0].predict(xte_lo[0], verbose=0), yte_lo[0])
print_metrics('lo','MACCS     ', res_lo[1].predict(xte_lo[1], verbose=0), yte_lo[1])
print_metrics('lo','Avalon    ', res_lo[2].predict(xte_lo[2], verbose=0), yte_lo[2])
print_metrics('lo','ECEP_MACCS', res_lo[3].predict(xte_lo[3], verbose=0), yte_lo[3])
print_metrics('lo','ECEP_Ava. ', res_lo[4].predict(xte_lo[4], verbose=0), yte_lo[4])
print_metrics('lo','MACCS_Ava.', res_lo[5].predict(xte_lo[5], verbose=0), yte_lo[5])
print_metrics('lo','ECFP+MACCS+Ava.', res_lo[6].predict(xte_lo[6], verbose=0), yte_lo[6])

# [data : lo ][model : ECEP       ] = r2 : 0.62102, mae : 0.73440, mse : 1.00307, rmse : 1.00153
# [data : lo ][model : MACCS      ] = r2 : 0.59578, mae : 0.68067, mse : 1.06987, rmse : 1.03435
# [data : lo ][model : Avalon     ] = r2 : 0.68422, mae : 0.67421, mse : 0.83580, rmse : 0.91422
# [data : lo ][model : ECEP_MACCS ] = r2 : 0.68492, mae : 0.64230, mse : 0.83395, rmse : 0.91321
# [data : lo ][model : ECEP_Ava.  ] = r2 : 0.69790, mae : 0.64407, mse : 0.79957, rmse : 0.89419
# [data : lo ][model : MACCS_Ava. ] = r2 : 0.70558, mae : 0.61507, mse : 0.77926, rmse : 0.88276
# [data : lo ][model : ECFP+MACCS+Ava. ] = r2 : 0.71277, mae : 0.62492, mse : 0.76024, rmse : 0.87192

In [ ]:
print_metrics('hu','ECEP      ', res_hu[0].predict(xte_hu[0], verbose=0), yte_hu[0])
print_metrics('hu','MACCS     ', res_hu[1].predict(xte_hu[1], verbose=0), yte_hu[1])
print_metrics('hu','Avalon    ', res_hu[2].predict(xte_hu[2], verbose=0), yte_hu[2])
print_metrics('hu','ECEP_MACCS', res_hu[3].predict(xte_hu[3], verbose=0), yte_hu[3])
print_metrics('hu','ECEP_Ava. ', res_hu[4].predict(xte_hu[4], verbose=0), yte_hu[4])
print_metrics('hu','MACCS_Ava.', res_hu[5].predict(xte_hu[5], verbose=0), yte_hu[5])
print_metrics('hu','ECFP+MACCS+Ava.', res_hu[6].predict(xte_hu[6], verbose=0), yte_hu[6])

# [data : hu ][model : ECEP       ] = r2 : 0.74465, mae : 0.79418, mse : 1.13364, rmse : 1.06472
# [data : hu ][model : MACCS      ] = r2 : 0.81311, mae : 0.67737, mse : 0.82969, rmse : 0.91088
# [data : hu ][model : Avalon     ] = r2 : 0.83078, mae : 0.64302, mse : 0.75128, rmse : 0.86676
# [data : hu ][model : ECEP_MACCS ] = r2 : 0.84665, mae : 0.61117, mse : 0.68083, rmse : 0.82512
# [data : hu ][model : ECEP_Ava.  ] = r2 : 0.85118, mae : 0.58893, mse : 0.66071, rmse : 0.81284
# [data : hu ][model : MACCS_Ava. ] = r2 : 0.85636, mae : 0.57487, mse : 0.63769, rmse : 0.79856
# [data : hu ][model : ECFP+MACCS+Ava. ] = r2 : 0.86441, mae : 0.57482, mse : 0.60195, rmse : 0.77585

In [37]:
ypred_de = [res_de[0].predict(xte_de[0], verbose=0),
            res_de[1].predict(xte_de[1], verbose=0),
            res_de[2].predict(xte_de[2], verbose=0),
            res_de[3].predict(xte_de[3], verbose=0),
            res_de[4].predict(xte_de[4], verbose=0),
            res_de[5].predict(xte_de[5], verbose=0),
            res_de[6].predict(xte_de[6], verbose=0)]

In [38]:
ypred_ws = [res_ws[0].predict(xte_ws[0], verbose=0),
            res_ws[1].predict(xte_ws[1], verbose=0),
            res_ws[2].predict(xte_ws[2], verbose=0),
            res_ws[3].predict(xte_ws[3], verbose=0),
            res_ws[4].predict(xte_ws[4], verbose=0),
            res_ws[5].predict(xte_ws[5], verbose=0),
            res_ws[6].predict(xte_ws[6], verbose=0)]

ypred_de = [res_de[0].predict(xte_de[0], verbose=0),
            res_de[1].predict(xte_de[1], verbose=0),
            res_de[2].predict(xte_de[2], verbose=0),
            res_de[3].predict(xte_de[3], verbose=0),
            res_de[4].predict(xte_de[4], verbose=0),
            res_de[5].predict(xte_de[5], verbose=0),
            res_de[6].predict(xte_de[6], verbose=0)]

ypred_lo = [res_lo[0].predict(xte_lo[0], verbose=0),
            res_lo[1].predict(xte_lo[1], verbose=0),
            res_lo[2].predict(xte_lo[2], verbose=0),
            res_lo[3].predict(xte_lo[3], verbose=0),
            res_lo[4].predict(xte_lo[4], verbose=0),
            res_lo[5].predict(xte_lo[5], verbose=0),
            res_lo[6].predict(xte_lo[6], verbose=0)]

ypred_hu = [res_hu[0].predict(xte_hu[0], verbose=0),
            res_hu[1].predict(xte_hu[1], verbose=0),
            res_hu[2].predict(xte_hu[2], verbose=0),
            res_hu[3].predict(xte_hu[3], verbose=0),
            res_hu[4].predict(xte_hu[4], verbose=0),
            res_hu[5].predict(xte_hu[5], verbose=0),
            res_hu[6].predict(xte_hu[6], verbose=0)]

In [ ]:
def vis_res_fingerprint(ypre,yte,name,target_path):
    fig = plt.figure(constrained_layout=True, figsize=(15, 5))
    fig.suptitle('{0} Fingerprint Compare.'.format(name), fontsize=16)
    ax0 = fig.add_subplot(131)
    ax0.set_title('ECFP, MACCS, Avalon')
    ax0.plot(yte[0],yte[0],'k-')
    ax0.plot(yte[0],ypre[0],'b*',label='%s R2 = %.3f %%' % ('ECFP     ',r2_score(yte[0],ypre[0])))
    ax0.plot(yte[1],ypre[1],'r*',label='%s R2 = %.3f %%' % ('MACCS  ',r2_score(yte[1],ypre[1])))
    ax0.plot(yte[2],ypre[2],'c*',label='%s R2 = %.3f %%' % ('Avalon  ',r2_score(yte[2],ypre[2])))
    ax0.legend(loc='upper left')
    ax0.set_ylabel('Y Truth')
    ax0.set_xlabel('Y Prediction')
    
    ax1 = fig.add_subplot(132)
    ax1.set_title('ECFP+MACCS, ECFP+Avalon, MACCS+Avalon')
    ax1.plot(yte[0],yte[0],'k-')
    ax1.plot(yte[3],ypre[3],'b*',label='%s R2 = %.3f %%' % ('ECEP_MACCS   ',r2_score(yte[3],ypre[3])))
    ax1.plot(yte[4],ypre[4],'r*',label='%s R2 = %.3f %%' % ('ECEP_Avalon    ',r2_score(yte[4],ypre[4])))
    ax1.plot(yte[5],ypre[5],'c*',label='%s R2 = %.3f %%' % ('MACCS_Avalon ',r2_score(yte[5],ypre[5])))
    ax1.legend(loc='upper left')
    ax1.set_ylabel('Y Truth')
    ax1.set_xlabel('Y Prediction')
    
    ax2 = fig.add_subplot(133)
    ax2.set_title('ECFP+MACCS+Avalon Fingerprints')
    ax2.plot(yte[0],yte[0],'k-')
    ax2.plot(yte[6],ypre[6],'b*',label='%s R2 = %.3f %%' % ('ECFP+MACCS+Avalon.  ',r2_score(yte[6],ypre[6])))
    ax2.legend(loc='upper left')
    ax2.set_ylabel('Y Truth')
    ax2.set_xlabel('Y Prediction')
    plt.show()
    fig.savefig(f"{target_path}/fingerprint_compare_{name}.png", dpi=300)
vis_res_fingerprint(ypred_ws,yte_ws,'ws496',target_path)
vis_res_fingerprint(ypred_de,yte_de,'delaney',target_path)
vis_res_fingerprint(ypred_lo,yte_lo,'lovrics',target_path)
vis_res_fingerprint(ypred_hu,yte_hu,'huuskonen',target_path)